Changes : 

* From (1) S12's don't match. Unclear what the issue is. The highest transmission is when there seems to be strong higher order in the waveguide -- issue with eigenmode decomposition? 
* For simplicity, since we know S12 = S21 should hold for LTI system, we'll just source from the waveguide. Also, just look at diretionality for now and compare with fiber mode later.

So in compute, remove  the look over sources and hardcode 1 = waveguide.
In the script, remove all references to fiber, and add the near-2-far monitor. Remove eigemode measurements.

In [ ]:
from gc_outcoupler2 import initialize

In [ ]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('-period', type=float, default=0.767723445279288, help='grating period (default: 0.767723445279288 um)')
parser.add_argument('-FF', type=float, default=0.5, help='Fill factor (default: 0.5 um)')
parser.add_argument('-theta', type=float, default=8, help='fiber_angle (default: 8 degrees)')    
parser.add_argument('-x', type=float, default=1, help='Fiber position (default: 1 um)')
parser.add_argument('-source', type=int, default=0, help='1 for waveguide, 0 for fiber')   
parser.add_argument('-filename', type=str, default='default', help='data name')  
args = parser.parse_args()
sim, neafielf, monitor = initialize(args)

In [ ]:
import matplotlib.pyplot as plt

f = plt.figure(dpi=100)
sim.plot2D(ax=f.gca())
plt.show()

In [ ]:
!mpirun -np 8 python gc_outcoupler2.py -period 0.5 -FF 0.5 -theta 8 -x 1 -source 1 -filename 'test' > ./logs/test.log

In [ ]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('./data') if isfile(join('./data', f))]

In [ ]:
onlyfiles[0]

In [ ]:
import pickle

dbfile = open('./data/' + onlyfiles[0], 'rb')
db = pickle.load(dbfile)
dbfile.close()

In [ ]:
db

In [ ]:
import numpy as np

farfield_power = db['farfield_power']
farfield_angles = db['farfield_angles']

Ex=farfield_power[:,0]
Ey=farfield_power[:,1]
Ez=farfield_power[:,2]
Hx=farfield_power[:,3]
Hy=farfield_power[:,4]
Hz=farfield_power[:,5]
Ex=np.conj(Ex)
Ey=np.conj(Ey)
Ez=np.conj(Ez)
Px=np.real(np.multiply(Ey,Hz)-np.multiply(Ez,Hy))
Py=np.real(np.multiply(Ez,Hx)-np.multiply(Ex,Hz))
Pz=np.real(np.multiply(Ex,Hy)-np.multiply(Ey,Hx))
Pr=np.sqrt(np.square(Px)+np.square(Py))
Pnorm = Pr/np.max(Pr)

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(farfield_angles, Pnorm)
ax.set_rmax(1)
ax.set_rticks([0.25, 0.5, 0.75, 1])  # Less radial ticks
ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
ax.grid(True)

# ax.set_title("A line plot on a polar axis", va='bottom')
plt.show()

In [ ]:
1/64

In [ ]:
import numpy as np
np.linspace(0.1, 0.9, 9)

Maybe we should monitor the outgoing power too. Let's add a mode monitor again. (reload). Also gives reflection.